## Dicionário de Dados - Conectividade Anatel:

- ano = Ano da observação
- sigla_uf = Sigla da Unidade Federativa (UF) do Brasil
- id_municipio = Código IBGE do município
- ibc = Índice Brasileiro de Conectividade
- cobertura_pop_4g5g = Cobertura populacional de 4G e 5G (percentual de moradores cobertos com sinal de tecnologia móvel; 2021: 3G+4G, 2022 em diante: 4G+5G)
- fibra = Presença de backhaul de rede de fibra ótica no município; 0 indica ausência total, 50 condição intermediária e 100 cobertura máxima
- densidade_smp = Densidade de acessos de Serviço Móvel Pessoal (telefonia móvel) — número de acessos dividido pela população, ponderado pela tecnologia (2G, 3G, 4G e 5G)
- hhi_smp = Índice de competitividade (HHI) das empresas de telefonia móvel no município, que mensura o grau de concentração do setor
- densidade_scm = Densidade de acessos de Serviços de Comunicação Multimídia (internet banda larga) — número de acessos dividido pela população, ponderado pela faixa de velocidade
- hhi_scm = Índice de competitividade (HHI) dos provedores de banda larga no município, que mensura o grau de concentração do setor
- adensamento_estacoes = Adensamento de estações de telefonia móvel

**Fonte:** https://basedosdados.org/dataset/ad45c5dc-ecc6-43db-ae2c-45d71939e7c5?table=c7166975-1f44-4c54-852a-80a3d22004a4

## Métricas 2024 conectividade de internet

In [0]:
CREATE OR REPLACE TABLE data_lake_turing_.prata.metricas_conectividade_uf (
WITH metricas_conectividade AS (
SELECT 
ano,
sigla_uf,
ROUND((MAX(ibc) - MIN(ibc)),2) AS Amplitude_ibc,
ROUND(AVG(ibc),2) AS media_ibc,
ROUND(AVG(cobertura_pop_4g5g),2) AS media_cobertura,
ROUND(AVG(fibra),0) AS media_fibra,
ROUND(AVG(densidade_smp),2) AS media_densidade_smp,
ROUND(AVG(hhi_smp),2) AS media_hhi_smp,
ROUND(AVG(densidade_scm),2) AS media_densidade_scm,
ROUND(AVG(hhi_scm),2) AS media_hhi_scm
FROM data_lake_turing_.bronze.anatel_indice_brasileiro_conectividade_municipio
WHERE ano = 2024
GROUP BY 1,2)

SELECT 
*,
CASE 
  WHEN media_fibra = 0 THEN 'ausencia_total'
  WHEN media_fibra >=0 and media_fibra <=50 THEN 'intermediaria'
  WHEN media_fibra >50 THEN 'cobertura_maxima'
  ELSE 'sem_informacoes'
END AS cobertura_fibra
FROM metricas_conectividade)

## Crescimento e evolução da conectividade

In [0]:
CREATE OR REPLACE TABLE data_lake_turing_.prata.evolucao_conectividade (
WITH metricas_conectividade AS (
  SELECT 
  ano,
  sigla_uf,
  max(ibc) AS ibc
  FROM data_lake_turing_.bronze.anatel_indice_brasileiro_conectividade_municipio
  GROUP BY 1,2),
  
  crescimento AS (
    SELECT
        sigla_uf,
        ano,
        ibc,
        LAG(ibc) OVER (PARTITION BY sigla_uf ORDER BY ano) AS ibc_anterior,
        ROUND(
            (ibc - LAG(ibc) OVER (PARTITION BY sigla_uf ORDER BY ano)) 
            / LAG(ibc) OVER (PARTITION BY sigla_uf ORDER BY ano) * 100, 2
        ) AS crescimento_percentual
    FROM metricas_conectividade
),
media_uf AS (
    SELECT
        sigla_uf,
        AVG(ibc) AS avg_ibc,
        AVG(crescimento_percentual) AS avg_crescimento
    FROM crescimento
    GROUP BY sigla_uf
)
SELECT
    NTILE(4) OVER (ORDER BY avg_ibc) AS quartil,
    sigla_uf,
    avg_ibc,
    avg_crescimento
FROM media_uf
ORDER BY quartil, avg_ibc
)
